In [42]:
import numpy as np

size = 32
bitlist = np.random.randint(2,size=size)
#max number
#bitlist = np.ones(size, dtype=int)
#min number
#bitlist = np.zeros(size, dtype=int)

def bit_to_float(bitlist):
    res=0
    j=-1
    for i in range(0,len(bitlist)):
        x=2**j
        res+=x*bitlist[i]
        j-=1
    return res

def bit_to_float_double(bitlist, doms):
    res=[0,0]
    halflen = int(len(bitlist)/2)
    j=-1
    for i in range(0,halflen):
        x=2**j
        res[0] += x*bitlist[i]
        j-=1
    j=-1
    for i in range(halflen, 2*halflen):
        x=2**j
        res[1] += x*bitlist[i]
        j-=1
    return np.array(np.multiply(doms,res))

print(bit_to_float_double(bitlist, [2,2]))

[1.90896606 1.95269775]


In [41]:
np.array([1,2])**2

array([1, 4])

In [44]:
#function to optimize

def bit_to_float_double(bitlist, doms):
    res=[0,0]
    halflen = int(len(bitlist)/2)
    j=-1
    for i in range(0,halflen):
        x=2**j
        res[0] += x*bitlist[i]
        j-=1
    j=-1
    for i in range(halflen, 2*halflen):
        x=2**j
        res[1] += x*bitlist[i]
        j-=1
    return np.array(np.multiply(doms,res))


def ackley(xvec, a=20, b=0.2, c = 2*np.pi):
    d=len(xvec)
    func = -a*np.exp(-b*np.sqrt(sum(xvec**2)/d)) - np.exp(sum(np.cos(c*xvec))/d) + a + np.exp(1)
    return func

def evaluate_function(population):
    evals = np.zeros(len(population))
    for p in len(population):
        evals[p] = ackley(bit_to_float_double(population[p], [16,16]))
    return evals

def breeding(population, fidelity, selection_rate):
    sorted_fid = np.argsort(fidelity)
    #select N/2 breeding pairs
    N= len(population)
    nallele= len(population[0])
    npairs = int(N/2)
    prob_list = [(1 + ((N-k%N)/(N-1))*(selection_rate - 1))*2/((1+selection_rate)*N) for k in range(N)]
    #find all the pairs
    all_pairs = []
    #selected = [0,0]
    for n in range(npairs):
        cond = True
        while cond:
            selected = np.random.choice(N,2,p=prob_list)
            if selected[0] != selected[1]:
                cond = False
            else:
                cond = True

        pair = [sorted_fid[selected[0]], sorted_fid[selected[1]]]
        all_pairs.append(selected)

    
    #now, swap
    for n in range(npairs):
        #where to cut 
        cut_space = np.random.randint(nallele-1)
        n1 = all_pairs[n][0]
        n2 = all_pairs[n][1]
        cut_pair_1 = population[n1][cut_space:]
        cut_pair_2 = population[n2][cut_space:]
        population[n1][cut_space:] = cut_pair_2
        population[n2][cut_space:] = cut_pair_1
    
    return population

def mutation(population, mutation_rate):
    for n in range(len(population)):
        for j in range(len(population[n])):
            val = population[n][j]
            if np.random.rand() < mutation_rate:
                population[n][j] = (val + 1)%2
    return population


#population of N individuals
N=20
size = 32 #16 bit precision for each number
population = [np.random.randint(2,size=size) for j in range(N)]
mutation_rate = 0.05  #5 percent allele mutation
selection_rate = 3

iter = 0
maxiter = 1000
kept_fidelity = np.zeros((maxiter, 20))
while iter < maxiter:
    fidelity = evaluate_function(population)
    kept_fidelity[iter] = fidelity
    population = breeding(population, fidelity, selection_rate)
    population = mutation(population, mutation_rate)
    iter += 1



In [ ]:
#next: plot the fidelity results for each generations/member (20 lines on scatter)